In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [12]:

write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = '4467975.1'

file_list = ({'date': '28_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '28_09_2020', 'scene':'YMaze_LNovel','session':6,'scan':14},
             {'date': '29_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '01_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':2},
             {'date': '02_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '04_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '04_10_2020', 'scene':'YMaze_RewardReversal','session':2,'scan':10},
             {'date': '05_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3},
             {'date': '06_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':2},
             {'date': '06_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':5},
             )



In [13]:
def check_and_make_h5s(file_list):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        try:
            source_folder  = os.path.join('/media','mplitt','Backup Plus1','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
#         if not os.path.exists(h5path):
#             _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [19]:
ops = np.load(os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","ops.npy"),allow_pickle=True).all()

In [21]:
print(ops.keys())

dict_keys(['suite2p_version', 'look_one_level_down', 'fast_disk', 'delete_bin', 'mesoscan', 'bruker', 'h5py', 'h5py_key', 'save_path0', 'save_folder', 'subfolders', 'move_bin', 'nplanes', 'nchannels', 'functional_chan', 'tau', 'fs', 'force_sktiff', 'frames_include', 'multiplane_parallel', 'preclassify', 'save_mat', 'save_NWB', 'combined', 'aspect', 'do_bidiphase', 'bidiphase', 'bidi_corrected', 'do_registration', 'two_step_registration', 'keep_movie_raw', 'nimg_init', 'batch_size', 'maxregshift', 'align_by_chan', 'reg_tif', 'reg_tif_chan2', 'subpixel', 'smooth_sigma_time', 'smooth_sigma', 'th_badframes', 'norm_frames', 'force_refImg', 'pad_fft', 'nonrigid', 'block_size', 'snr_thresh', 'maxregshiftNR', '1Preg', 'spatial_hp', 'spatial_hp_reg', 'spatial_hp_detect', 'pre_smooth', 'spatial_taper', 'roidetect', 'spikedetect', 'anatomical_only', 'sparse_mode', 'diameter', 'spatial_scale', 'connected', 'nbinned', 'max_iterations', 'threshold_scaling', 'max_overlap', 'high_pass', 'use_builtin_c

In [27]:
print(ops['save_path0'])
print(ops['save_path'])
print(ops['ops_path'])
print(ops['reg_file'])
print(ops['reg_file_chan2'])
ops['save_path0'] = os.path.join(write_folder,mouse,"YMazeAll")
ops['save_path'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0")
ops['ops_path'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","ops.npy")
ops['reg_file'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","data.bin")
ops['reg_file_chan2'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","data_chan2.bin")
np.save(ops['ops_path'],ops)

/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll
/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll/suite2p/plane0
/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll/suite2p/plane0/ops.npy
/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll/suite2p/plane0/data.bin
/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll/suite2p/plane0/data_chan2.bin


In [28]:
h5list_all, frs_all = check_and_make_h5s(file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': h5list_all,
                               'save_path0': os.path.join(write_folder,mouse,"YMazeAll"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':frs_all[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'saveNWB':True,
                                'threshold_scaling':.75,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 6, 'scan': 14}
{'date': '29_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '30_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4}
{'date': '01_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}
{'date': '02_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '03_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '04_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '04_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 10}
{'date': '05_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3}
{'date': '06_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 2}
{'date': '06_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 5}
['/mnt/BigDisk/2P_scratch/4467975.1/28_09_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/mnt/BigDisk/2

In [9]:
_file_list = ({'date': '29_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '01_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':2},
             {'date': '02_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '05_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3},
             )


for f in _file_list:
    _h5list,_frs = check_and_make_h5s([f,])
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':.75,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)

{'date': '29_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
['/media/mplitt/BigDisk/2P_scratch/4467975.1/29_09_2020/YMaze_LNovel/YMaze_LNovel_001_003']
{}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/media/mplitt/BigDisk/2P_scratch/4467975.1/29_09_2020/YMaze_LNovel/YMaze_LNovel_001_003/YMaze_LNovel_001_003.h5']
time 817.62 sec. Wrote 38344 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 38344 frames
NOTE: estimated bidiphase offset from data: 0 pixels
Reference frame, 17.83 sec.
Registered 2000/38344 in 79.11s
Registered 4000/38344 in 157.84s
Registered 6000/38344 in 237.95s
Registered 8000/38344 in 324.15s
Registered 10000/38344 in 414.60s
Registered 12000/38344 in 504.90s
Registered 14000/38344 in 594.55s
Registered 16000/38344 in 684.72s
Registered 18

In [10]:
_file_list = ({'date': '28_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '28_09_2020', 'scene':'YMaze_LNovel','session':6,'scan':14},)

_h5list,_frs = check_and_make_h5s(_file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"28_09_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2})
ops=s2p.run_s2p(ops=ops)

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 6, 'scan': 14}
['/media/mplitt/BigDisk/2P_scratch/4467975.1/28_09_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/media/mplitt/BigDisk/2P_scratch/4467975.1/28_09_2020/YMaze_LNovel/YMaze_LNovel_006_014']
{}
h5
** Found 2 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/media/mplitt/BigDisk/2P_scratch/4467975.1/28_09_2020/YMaze_LNovel/YMaze_LNovel_001_003/YMaze_LNovel_001_003.h5', '/media/mplitt/BigDisk/2P_scratch/4467975.1/28_09_2020/YMaze_LNovel/YMaze_LNovel_006_014/YMaze_LNovel_006_014.h5']
time 704.78 sec. Wrote 34199 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 34199 frames
NOTE: estimated bidiphase offset from data: 0 pixels
Reference frame, 16.28 sec.


In [11]:
_file_list = ({'date': '04_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '04_10_2020', 'scene':'YMaze_RewardReversal','session':2,'scan':10},)


_h5list,_frs = check_and_make_h5s(_file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"04_12_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2})
ops=s2p.run_s2p(ops=ops)


{'date': '04_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '04_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 10}
['/media/mplitt/BigDisk/2P_scratch/4467975.1/04_10_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/media/mplitt/BigDisk/2P_scratch/4467975.1/04_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_002_010']
{}
h5
** Found 2 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/media/mplitt/BigDisk/2P_scratch/4467975.1/04_10_2020/YMaze_LNovel/YMaze_LNovel_001_003/YMaze_LNovel_001_003.h5', '/media/mplitt/BigDisk/2P_scratch/4467975.1/04_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_002_010/YMaze_RewardReversal_002_010.h5']
time 786.01 sec. Wrote 37536 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 37536 frames
NOTE: estimated bidiphase offset 

In [12]:
_file_list = ({'date': '06_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':2},
             {'date': '06_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':5})

_h5list,_frs = check_and_make_h5s(_file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"06_12_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2})
ops=s2p.run_s2p(ops=ops)


{'date': '06_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 2}
{'date': '06_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 5}
['/media/mplitt/BigDisk/2P_scratch/4467975.1/06_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_002', '/media/mplitt/BigDisk/2P_scratch/4467975.1/06_10_2020/YMaze_LNovel/YMaze_LNovel_001_005']
{}
h5
** Found 2 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/media/mplitt/BigDisk/2P_scratch/4467975.1/06_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_002/YMaze_RewardReversal_001_002.h5', '/media/mplitt/BigDisk/2P_scratch/4467975.1/06_10_2020/YMaze_LNovel/YMaze_LNovel_001_005/YMaze_LNovel_001_005.h5']
time 774.81 sec. Wrote 37347 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 37347 frames
NOTE: estimated bidiphase offset f

In [13]:
for h5 in h5list_all:
    !rm {h5}/*.h5 